In [1]:
import windrose
from windrose import WindroseAxes
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import numpy as np
import mpl_toolkits
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import netCDF4 as nc
import metpy
import metpy.calc as mpcalc
from metpy.units import units, pandas_dataframe_to_unit_arrays
from metpy.plots import ImagePlot, MapPanel, PanelContainer
from metpy.calc import wind_components
from metpy.interpolate import cross_section
from scipy.constants import convert_temperature
import xarray as xr
import math
import matplotlib
import os
import plotly
import plotly.express as px
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
#Import Data 

BeagleSprings=nc.Dataset('beaglesprings.nc','r') 
CatCreek=nc.Dataset('catcreek.nc','r') 
Challis=nc.Dataset('challisairport.nc','r') 
EzraCreek=nc.Dataset('ezracreek.nc','r') 
HillCity=nc.Dataset('hillcity.nc','r') 
HiltsCreek=nc.Dataset('hiltscreek.nc','r') 
LemhiBasin=nc.Dataset('lemhibasin.nc','r') 
Lemhi_North=nc.Dataset('lemhinorth.nc','r') 
Lemhi_NSfc=nc.Dataset('lemhinorthsurface.nc','r') 
Lemhi_Ridge=nc.Dataset('lemhiridge.nc','r') 
Lemhi_South=nc.Dataset('lemhisouth.nc','r') 
Lemhi_SSfc=nc.Dataset('lemhisouthsurface.nc','r') 
MeadowLake=nc.Dataset('meadowlake.nc','r') 
Moonshine=nc.Dataset('moonshine.nc','r') 
MorganCreek=nc.Dataset('morgancreek.nc','r') 
Pahsimeroi_Basin=nc.Dataset('pahsimeroibasin.nc','r') 
Pahsimeroi_North=nc.Dataset('pahsimeroinorth.nc','r')
Pahsimeroi_South=nc.Dataset('pahsimeroisouth.nc','r') 
Pahsimeroi_Sfc=nc.Dataset('pahsimeroisurface.nc','r') 


In [3]:
#Data Files

files=[
    {'filepath':'beaglesprings.nc','name':'Beagle Springs'},
    {'filepath':'catcreek.nc','name':'Cat Creek'},
    {'filepath':'challisairport.nc','name':'Challis Airport'},
    {'filepath':'ezracreek.nc','name':'Ezra Creek'},
    {'filepath':'hillcity.nc','name':'Hill City'},
    {'filepath':'hiltscreek.nc','name':'Hilts Creek'},
    {'filepath':'lemhibasin.nc','name':'Lemhi Basin'},
    {'filepath':'lemhinorth.nc','name':'Lemhi North'},
    {'filepath':'lemhinorthsurface.nc','name':'Lemhi North Surface'},
    {'filepath':'lemhiridge.nc','name':'Lemhi Ridge'},
    {'filepath':'lemhisouth.nc','name':'Lemhi South'},
    {'filepath':'lemhisouthsurface.nc','name':'Lemhi South Surface'},
    {'filepath':'meadowlake.nc','name':'Meadow Lake'},
    {'filepath':'moonshine.nc','name':'Moonshine'},
    {'filepath':'morgancreek.nc','name':'Morgan Creek'},
    {'filepath':'pahsimeroibasin.nc','name':'Pahsimeroi Basin'},
    {'filepath':'pahsimeroinorth.nc','name':'Pahsimeroi North'},
    {'filepath':'pahsimeroisouth.nc','name':'Pahsimeroi South'},
    {'filepath':'pahsimeroisurface.nc','name':'Pahsimeroi Surface'}
    ]

In [54]:
#Plot Windroses
    
def make_windrose(filepath=None, name=None):
    filepath = filepath if os.path.exists(filepath) else os.path.join('windrose_data', filepath)
    ds = xr.open_dataset(filepath, engine = 'netcdf4')
    df = ds[['U_700MB','V_700MB']].to_dataframe()
    df.U_700MB[df.U_700MB > 999] = np.nan
    df.V_700MB[df.V_700MB > 999] = np.nan
    df['WSPD'] = np.sqrt(df.U_700MB**2+df.V_700MB**2)
    df['WDIR'] = np.arctan2(df.U_700MB.values,df.V_700MB.values)*180/math.pi+180;
    df.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
    ax = WindroseAxes.from_ax()
    ax.bar(df.WDIR, df.WSPD, normed=True, opening=0.8, edgecolor='white')
    ax.set_legend()
    ax.set_title(name)
    ax.legend(title='Wind Speed (m/s)',loc = 1)
    #matplotlib.pyplot.savefig(os.path.join('windrose_plots',"{}_WR.png".format(name.replace(' ','_'))), dpi=300, transparent=False)
    matplotlib.pyplot.close()
    
_=[make_windrose(**dd) for dd in files]


In [55]:
#Plot windroses by precipitation

def make_windrose(filepath=None, name=None):
    filepath = filepath if os.path.exists(filepath) else os.path.join('windrose_data', filepath)
    ds = xr.open_dataset(filepath, engine = 'netcdf4')
    df = ds[['U_700MB','V_700MB','PREC_ACC_NC']].to_dataframe()
    df=df[df['PREC_ACC_NC']>0]
    df.U_700MB[df.U_700MB > 999] = np.nan
    df.V_700MB[df.V_700MB > 999] = np.nan
    df.PREC_ACC_NC[df.PREC_ACC_NC > 0.01] = np.nan
    df['WSPD'] = np.sqrt(df.U_700MB**2+df.V_700MB**2)
    df['WDIR'] = np.arctan2(df.U_700MB.values,df.V_700MB.values)*180/math.pi+180;
    df.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
    
    ax = WindroseAxes.from_ax()
    ax.bar(df.WDIR, df.WSPD, normed=True, opening=0.8, edgecolor='white')
    ax.set_legend()
    ax.set_title(name)
    ax.legend(title='Wind Speed (m/s)',loc = 1)
    #matplotlib.pyplot.savefig(os.path.join('windrose_precip_plots',"{}_WR.png".format(name.replace(' ','_'))), dpi=300, transparent=False)
    matplotlib.pyplot.close()
    
_=[make_windrose(**dd) for dd in files]


Frequency of Wind Direction/Speed and Total Precipitation

In [32]:


#filepath = filepath if os.path.exists(filepath) else os.path.join('windrose_data', filepath)
filepath=os.path.join('windrose_data', 'beaglesprings.nc')
ds = xr.open_dataset(filepath, engine = 'netcdf4')
df = ds[['U_700MB','V_700MB','PREC_ACC_NC']].to_dataframe()
df.U_700MB[df.U_700MB > 999] = np.nan
df.V_700MB[df.V_700MB > 999] = np.nan
df['WSPD'] = np.sqrt(df.U_700MB**2+df.V_700MB**2)
df['WDIR'] = np.arctan2(df.U_700MB.values,df.V_700MB.values)*180/math.pi+180;
df.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
df=df[df['PREC_ACC_NC']>0.01]

mean_df=df.groupby(pd.Grouper(freq='M')).mean()
#prec_trace=go.Bar(x=mean_df.index, y=mean_df['PREC_ACC_NC'],yaxis="y" )
dir_trace=go.Bar(x=mean_df.index, y=mean_df['WDIR'],yaxis="y2", marker=dict(color='darkorange'))
spd_trace=go.Bar(x=mean_df.index, y=mean_df['WSPD'],xaxis="x3",yaxis="y3",marker=dict(color='mediumvioletred'))

sum_df=df.groupby(pd.Grouper(freq='M')).sum()
prec_trace=go.Bar(x=sum_df.index, y=sum_df['PREC_ACC_NC'],yaxis="y", marker=dict(color='lightseagreen'))


layout = go.Layout(
   
    xaxis3=dict(
        title='Year'
    ),
    yaxis=dict(
        title='(mm)'
    ),
    yaxis2=dict(
        title='(Degrees)'
    ),
    yaxis3=dict(
        title='(m/s)'
    )
)

# data=[prec_trace, dir_trace, spd_trace]
# fig = go.Figure(data=data, layout=layout)

fig = make_subplots(rows=3, cols=1, shared_xaxes=True,subplot_titles=('Total Precipitation','Mean Wind Direction', 'Mean Wind Speed'))

fig.add_trace(prec_trace, 1, 1,)
fig.add_trace(dir_trace, 2, 1)
fig.add_trace(spd_trace, 3, 1)

fig.update_layout(layout, title = 'Beagle Springs', showlegend = False, height = 600)

fig.update_layout(yaxis=dict(tickvals=[0,10,20,30,40,50,60]),
                 yaxis2=dict(tickvals=[0,60,120,180,240,300,360]),
                 yaxis3=dict(tickvals=[0,5,10,15,20]),
                 xaxis3_tickformat = '%Y',
                 xaxis3=dict(tickvals=[2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013])
                 )
            

fig.show()
fig.write_image("BeagleSprings_Graph.png")

In [33]:

def doodoo_bee_sunglasses(filepath=None, name=None):
    filepath = filepath if os.path.exists(filepath) else os.path.join('windrose_data', filepath)
    # filepath=os.path.join('windrose_data', 'beaglesprings.nc')
    ds = xr.open_dataset(filepath, engine = 'netcdf4')
    df = ds[['U_700MB','V_700MB','PREC_ACC_NC']].to_dataframe()
    df.U_700MB[df.U_700MB > 999] = np.nan
    df.V_700MB[df.V_700MB > 999] = np.nan
    df['WSPD'] = np.sqrt(df.U_700MB**2+df.V_700MB**2)
    df['WDIR'] = np.arctan2(df.U_700MB.values,df.V_700MB.values)*180/math.pi+180;
    df.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
    df=df[df['PREC_ACC_NC']>0.01]
    #Mean Wind Direction and Spead 
    mean_df=df.groupby(pd.Grouper(freq='M')).mean()
    dir_trace=go.Bar(x=mean_df.index, y=mean_df['WDIR'],yaxis="y2", marker=dict(color='darkorange'))
    spd_trace=go.Bar(x=mean_df.index, y=mean_df['WSPD'],xaxis="x3",yaxis="y3",marker=dict(color='mediumvioletred'))
    #Precipitation Total 
    sum_df=df.groupby(pd.Grouper(freq='M')).sum()
    prec_trace=go.Bar(x=sum_df.index, y=sum_df['PREC_ACC_NC'],yaxis="y", marker=dict(color='lightseagreen'))
    layout = go.Layout(xaxis3=dict(title='Year'),yaxis=dict(title='(mm)'),yaxis2=dict(title='(Degrees)'),yaxis3=dict(title='(m/s)'))
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True,subplot_titles=('Total Precipitation','Mean Wind Direction', 'Mean Wind Speed'))
    fig.add_trace(prec_trace, 1, 1,)
    fig.add_trace(dir_trace, 2, 1)
    fig.add_trace(spd_trace, 3, 1)
    # fig.update_layout(layout, title = 'Beagle Springs', showlegend = False, height = 600)
    fig.update_layout(layout, title = name, showlegend = False, height = 600)
    fig.update_layout(yaxis=dict(tickvals=[0,10,20,30,40,50,60]),
                    yaxis2=dict(tickvals=[0,60,120,180,240,300,360]),
                    yaxis3=dict(tickvals=[0,5,10,15,20]),
                    xaxis3_tickformat = '%Y',
                    xaxis3=dict(tickvals=[2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013])
                    )
    fig.show()
    # fig.write_image("BeagleSprings_Graph.png")
    fig.write_image("{}_Graph.png".format(name.replace(' ','')))


_=[doodoo_bee_sunglasses(**dd) for dd in files]
    


In [34]:
import plotly.express as px
df = px.data.wind()
df

,direction,strength,frequency
0,N,0-1,0.5
1,NNE,0-1,0.6
2,NE,0-1,0.5
3,ENE,0-1,0.4
4,E,0-1,0.4
...,...,...,...
123,WSW,6+,0.1
124,W,6+,0.9
125,WNW,6+,2.2
126,NW,6+,1.5


In [36]:
df[df['direction']=='WNW']


,direction,strength,frequency
13,WNW,0-1,0.5
29,WNW,1-2,2.6
45,WNW,2-3,1.7
61,WNW,3-4,1.2
77,WNW,4-4,1.0
93,WNW,4-5,0.9
109,WNW,5-6,0.7
125,WNW,6+,2.2


In [ ]:
def precip_windrose(filepath=None, name=None):
    filepath = filepath if os.path.exists(filepath) else os.path.join('windrose_data', filepath)
    ds = xr.open_dataset(filepath, engine = 'netcdf4')
    df = ds[['U_700MB','V_700MB','PREC_ACC_NC']].to_dataframe()
    df=df[df['PREC_ACC_NC']>0]
    df.U_700MB[df.U_700MB > 999] = np.nan
    df.V_700MB[df.V_700MB > 999] = np.nan
    df.PREC_ACC_NC[df.PREC_ACC_NC > 0.01] = np.nan
    df['WSPD'] = np.sqrt(df.U_700MB**2+df.V_700MB**2)
    df['WDIR'] = np.arctan2(df.U_700MB.values,df.V_700MB.values)*180/math.pi+180;
    df.replace([np.inf, -np.inf], np.nan).dropna(axis=0)

    

In [112]:
filepath = os.path.join('windrose_data', 'beaglesprings.nc')
# {'filepath':'beaglesprings.nc','name':'Beagle Springs'}
ds = xr.open_dataset(filepath, engine = 'netcdf4')
df = ds[['U_700MB','V_700MB','PREC_ACC_NC']].to_dataframe()
df=df[df['PREC_ACC_NC']>0]
df.U_700MB[df.U_700MB > 999] = np.nan
df.V_700MB[df.V_700MB > 999] = np.nan
df.PREC_ACC_NC[df.PREC_ACC_NC > 0.01] = np.nan
df['WSPD'] = np.sqrt(df.U_700MB**2+df.V_700MB**2)
df['WDIR'] = np.arctan2(df.U_700MB.values,df.V_700MB.values)*180/math.pi+180;
df.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
sum_df=df.groupby(pd.Grouper(freq='M')).sum()
mean_df=df.groupby(pd.Grouper(freq='M')).mean()
# sum_df=df.groupby(pd.Grouper(freq='W')).sum()
# mean_df=df.groupby(pd.Grouper(freq='W')).mean()
# mean_df['wadder']=sum_df['PREC_ACC_NC']
__df = pd.DataFrame({
    'spd':mean_df['WSPD'],
    'dir':mean_df['WDIR'],
    'prec':sum_df['PREC_ACC_NC']
    })

__df

,spd,dir,prec
Time,,,
2000-10-31,10.418925,194.052097,0.074621
2000-11-30,8.798519,239.655787,0.092014
2000-12-31,12.258482,257.705005,0.090257
2001-01-31,9.174407,224.162090,0.084926
2001-02-28,9.951766,202.227695,0.083877
...,...,...,...
2013-01-31,11.936021,244.160864,0.063075
2013-02-28,10.607302,225.415754,0.139147
2013-03-31,10.506950,254.317805,0.110267


In [113]:
sum_df

,U_700MB,V_700MB,PREC_ACC_NC,WSPD,WDIR
Time,,,,,
2000-10-31,182.850271,276.669079,0.074621,937.703282,17464.688767
2000-11-30,500.869050,-287.406785,0.092014,950.240010,25882.825013
2000-12-31,789.504427,-42.323421,0.090257,1017.453991,21389.515418
2001-01-31,411.053190,11.169278,0.084926,1036.707951,25330.316211
2001-02-28,701.145337,216.069269,0.083877,1253.922529,25480.689626
...,...,...,...,...,...
2013-01-31,914.366425,-454.390281,0.063075,1432.322483,29299.303692
2013-02-28,663.427986,-579.357427,0.139147,1262.268980,26824.474701
2013-03-31,882.176821,-136.345366,0.110267,1271.340945,30772.454368


In [116]:
fig = px.bar_polar(__df, r= 'spd', theta='dir',
                   color='prec', 
                   range_theta=[180-360,225-360],
                   width=10000,
                   height=10000
                # template="plotly_dark",
                # barmode='overlay',
                # barnorm='percent',
                  #  color_discrete_sequence= px.colors.sequential.Plasma_r
                   )
# fig.update_polars(sector=[__df['dir'].min()-360, __df['dir'].max()-360])

# fig = px.bar_polar(sum_df, r= 'WSPD', theta='WDIR',
#                    color='PREC_ACC_NC', 
#                 template="plotly_dark",
#                    color_discrete_sequence= px.colors.sequential.Plasma_r)

fig.show()